In [1]:
!pip install langchain
!pip install langchain[openai]
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [4]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 9.8 MB/s eta 0:00:00


In [5]:
from langchain_community.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [6]:
## Read the Pdfs from the folder
loader = PyPDFDirectoryLoader("us_census")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'us_census/acsbr-017.pdf', 'page': 0, 'page_label': '1'}, page_content='KEY DEFINITIONS\nHousehold income: Includes income of the \nhouseholder and all other people 15 years and \nolder in the household, whether or not they are \nrelated to the householder.\nMedian: The point that divides the household \nincome distribution into halves, one half with \nincome above the median and the other with \nincome below the median. The median is based \non the income distribution of all households, \nincluding those with no income.\nGini index: A summary measure of income \ninequality. The Gini index varies from 0 to 1, with \n0 indicating perfect equality, where there is a \nproportional distribution of income. A Gini index \nof 1 indicates perfect inequality, where one \nhousehold has all the income.\nHousehold Income in States and \nMetropolitan Areas: 2022\nAmerican Community Survey Briefs\nBy Kirby G. Posey\nACSBR-017\nDecember 2023\nINTRODUCTION\nThis brief pres

In [7]:
!pip install sentence_transformers

In [8]:
# Embeddings using HuggingFace
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-small-en-v1.5",
    # sentence-transformers/all-MiniLM-16-v2
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

<ipython-input-8-5fb2b0b25a49>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-2.70792041e-02 -4.59913388e-02 -2.82515138e-02 -4.11672294e-02
  3.41416188e-02  2.43200343e-02  9.57051516e-02 -2.61037052e-02
  3.80554907e-02  2.60476619e-02  5.47391102e-02 -5.11632226e-02
  1.97422355e-02  1.46766314e-02 -4.75789839e-03 -4.43913415e-03
 -2.72196531e-02  5.67295663e-02 -3.38217951e-02  3.77274654e-03
  1.59257188e-01 -3.34241912e-02 -4.65696044e-02 -1.08218668e-02
  1.51004210e-01  3.13124172e-02 -2.30167340e-03 -3.79209109e-02
 -4.02005501e-02 -1.46105319e-01  3.17057371e-02  2.80707330e-02
  2.45580506e-02  4.15470172e-03 -9.66768805e-03 -4.70971194e-04
  1.01747978e-02  5.48642278e-02 -1.24961315e-02 -5.47601888e-03
 -2.16648653e-02  2.44959607e-03  3.12912539e-02 -2.31652278e-02
 -6.11188933e-02  2.51090918e-02 -5.10432832e-02  4.56827134e-02
 -1.32516911e-02  3.73204090e-02 -7.05002621e-02  1.74874862e-04
 -3.12954858e-02  6.73781708e-02  1.99592635e-02 -4.08781581e-02
  7.21120462e-02 -1.90256219e-02  4.59805335e-04  4.12346199e-02
  5.59731759e-02 -5.65998

In [16]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.4 MB/s eta 0:00:00


In [17]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [18]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [19]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b4936e2ee90> search_kwargs={'k': 3}


In [24]:
import os
huggingfacehub_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [ ]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
llm = hf
llm.invoke(query)

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [ ]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
result = retrievalQA.invoke({"query": query})
print(result['result'])